
파이선 병렬프로그래밍: 수치 해석 예제 실습
===================================================


#### 2024.10.
### 한국과학기술정보연구원 강지훈

***

### 필요 패키지

  - mpi4py
  - numpy
  - random
  - scikit-learn
  - matplotlib

***


# 1. 벡터와 행렬 연산 (I)

## 1.1. 행렬/벡터 만들기

In [ ]:
!mkdir examples

In [20]:
import numpy as np

np.set_printoptions(linewidth=np.inf)

n = 10

A = np.random.rand(n, n)
B = np.random.rand(n, n)
v = np.random.rand(n)
w = np.random.rand(n)

np.save("examples/A", A)
np.save("examples/B", B)
np.save("examples/v", v)
np.save("examples/w", w)


## 1.2. 벡터 내적

1. 순차코드
   
   <img src = "images/image01.png">

2. 병렬코드 - 등분할

    <img src = "images/image02.png">

In [ ]:
%%writefile examples/v.py
import numpy as np
from mpi4py import MPI

np.set_printoptions(linewidth=np.inf,precision=3)

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    v = np.load("examples/v.npy")
    w = np.load("examples/w.npy")
    n = v.size
else :
    v = None
    w = None
    n = 0

n = comm.bcast(n, root = 0)


##### n_row 크기 정하기 
n_row = # FIX ME

##### 분할된 n_row 크기만큼 배열 생성 
v_row = np.empty(n_row, dtype = np.float64)
w_row = np.empty(n_row, dtype = np.float64)

##### Scatter 함수로 벡터 분할
comm.Scatter(v, v_row, root = 0) # FIX ME
comm.Scatter(w, w_row, root = 0) # FIX ME

##### 프로세스별 Local sum 
s = np.dot( # FIX ME

##### reduce를 이용한 Global sum
s_all = comm.allreduce( # FIX ME

#if rank == 1:
print(rank, s_all)


In [ ]:
! mpiexec -np 2 python examples/v.py

3. 병렬코드 - 비등분할

    <img src = "images/image03.png">

In [ ]:
%%writefile examples/v_var.py
import numpy as np
from mpi4py import MPI

##### 시작점과 끝접의 인덱스를 반환
def para_range(n, size, rank) :
    iwork = divmod(n, size) 
    ista = rank * iwork[0] + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    v = np.load("examples/v.npy")
    w = np.load("examples/w.npy")
    n = v.size
else :
    v = None
    w = None
    n = 0

##### 벡터의 전체 크기를 broadcast
n = comm.bcast(n, root = 0)

##### 프로세스별 범위 할당
ista, iend = # FIX ME
n_row = # FIX ME


##### Scatterv를 위해 n_row로부터 n_rows 리스트 생성
n_rows = comm.gather(n_row, root = 0)

v_row = np.empty(n_row, dtype = np.float64)
w_row = np.empty(n_row, dtype = np.float64)

##### n_rows 리스트를 이용하여 Scatter로 벡터의 비균등 할당
comm.Scatterv((v, n_rows), v_row, root = 0) #FIX ME
comm.Scatterv((w, n_rows), w_row, root = 0) #FIX ME

##### 분할된 벡터의 내적
s = np.dot(v_row,w_row)

##### reduce를 이용한 Global sum
s_all = comm.reduce( #FIX ME

if rank == 0:
    print(n_rows)
    print(s_all)


In [ ]:
! mpiexec -np 3 python examples/v_var.py

4. para_range 저장

In [ ]:
%%writefile examples/tools.py

def para_range(n, size, rank) :
    iwork = divmod(n, size) 
    ista = rank * iwork[0] + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend


## 1.3. 행렬-벡터곱

1. 순차코드
   
    <img src = "images/image04.png">

In [ ]:
A = np.load("examples/A.npy")
v = np.load("examples/v.npy")

b = np.matmul(A,v)
print (b)


2. 행렬의 행 등분할

    <img src = "images/image05.png">

In [ ]:
%%writefile examples/Av.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    v = np.load("examples/v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)
else :
    A = None
    n = 0
    n = comm.bcast(n, root = 0)
    v = np.empty(n, dtype = np.float64)

##### n_row 크기 정하기 
n_row = int(n / size)

##### n_row 만큰 부분배열 선언
A_row = np.empty((n_row, n), dtype = np.float64)

##### 행렬의 행 분할
comm.Scatter( #FIX ME

##### 벡터 v는 broadcast
comm.Bcast(v, root = 0)

##### 분할된 행렬과의 연산
b = np.matmul( #FIX ME

print(b, rank)

In [ ]:
! mpiexec -np 2 python examples/Av.py

3. 행렬의 행 비등분할

    <img src = "images/image06.png">

In [ ]:
%%writefile examples/Avar.py

# Matrix A의 Row decomposition

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    v = np.load("examples/v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)

else :
    A = None
    n = 0
    n = comm.bcast(n, root = 0)
    v = np.empty(n, dtype = np.float64)

##### 프로세스별 범위 할당
ista, iend = para_range(n, size, rank)

##### 프로세스별 행수 설정
n_row = (iend - ista + 1)

##### 분할된 행렬 선언
A_row = np.empty((n_row, n), dtype = np.float64)

##### Scatterv를 위해 n_row를 이용한 리스트 생성. 이 때 부분행렬 행수가 아닌 전체 크기를 계산 
n_rows = comm.gather(n_row * n, root = 0) #FIX ME

##### 행렬의 행 분할
comm.Scatterv((A, n_rows), A_row, root = 0) #FIX ME

comm.Bcast(v, root = 0)

b = np.matmul(A_row,v)

print(b, rank)

In [ ]:
! mpiexec -np 3 python examples/Avar.py

4. 행렬/벡터의 행 비등분할

    <img src = "images/image07_1.png">

- $w_0$계산을 위해 각 프로세스가 소유한 $v$들이 필요 
  
  <img src = "images/image07_2.png">

- 최종 형태

    <img src = "images/image07_3.png">

In [ ]:
%%writefile examples/Av_var.py

# Matrix A의 Row decomposition

from tools import para_range
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    v = np.load("examples/v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)

else :
    A = None
    v = None
    n = 0
    n = comm.bcast(n, root = 0)

##### 프로세스별 범위 할당
ista, iend = para_range(n, size, rank)

##### 프로세스별 행수 설정
n_row = (iend - ista + 1)

##### 분할된 행렬 선언
A_row = np.empty((n_row, n), dtype = np.float64)
v_row = np.empty(n_row, dtype = np.float64)

##### Scatterv를 위해 n_row를 이용한 리스트 생성. 이 때 부분행렬 행수가 아닌 전체 크기를 계산 
##### 분할된 벡터의 크기는 모든 프로세스가 알고 있어야 하므로 allgather를 이용
n_chunks = comm.gather(n_row * n, root = 0) #FIX ME
n_rows = comm.allgather(n_row) #FIX ME

##### 행렬과 벡터 분할
comm.Scatterv([A, n_chunks], A_row, root = 0) #FIX ME
comm.Scatterv([v, n_rows], v_row, root = 0) #FIX ME

##### 분할된 벡터 곱 범위 지정
vsta_list = []
vend_list = []

##### i번째 랭크가 곱해질 열 위치의 시작점과 끝점을 리스트 형태로 저장. 모든 랭크들이 계산
for i in range(size) :
    vsta_list.append(sum(n_rows[:i]))
    vend_list.append(sum(n_rows[:i])+n_rows[i])

##### Local MV (최초 자신의 벡터부분)
b = np.matmul(A_row[:,vsta_list[rank]:vend_list[rank]], v_row)

##### 송수신 프로세스 지정
inext = rank + 1 if rank < size - 1 else 0
iprev = rank - 1 if rank > 0 else size - 1

##### (전체 프로세스 크기 -1) 번만큼 통신을 수행하고 부분행렬-부분벡터 곱을 수행
for i in range(size - 1) :
##### 몇 번째 이전 랭크로부터 받았는지를 iloc을 이용하여 확인
    iloc = iprev - i if iprev >= i else iprev - i + size
    v_recv = np.empty(n_rows[iloc], dtype = np.float64)
##### 다음 랭크로 부분 벡터를 전달하며, 이전 랭크로부터 부분 벡터를 받음
    comm.Sendrecv(v_row, inext, 1, v_recv, iprev, 1) #FIX ME
##### 받은 벡터는 복사하여, 행렬-벡터 곱에 사용하는 한편, 다음 랭크로 전달할 수 있게 함
    v_row = np.copy(v_recv)
    b += np.matmul(A_row[:,vsta_list[iloc]:vend_list[iloc]], v_row)

print(b, rank)

In [ ]:
! mpiexec -np 3 python examples/Av_var.py